<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/NetworkDesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design

In [192]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [193]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Daten aus csv-Datei laden



In [194]:
! git clone https://github.com/AlexKressner/Industrielles_Management

fatal: destination path 'Industrielles_Management' already exists and is not an empty directory.


In [195]:
# Daten laden
import pandas as pd

In [196]:
folder = "Industrielles_Management/Daten/NetworkDesign"

In [197]:
path = f"{folder}/NetworkDesign_Invest_Kapa.csv"

In [198]:
kapa=pd.read_csv(path, sep=";")

In [199]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [200]:
path = f"{folder}/NetworkDesign_Produktionskosten.csv"

In [201]:
produktionskosten=pd.read_csv(path, sep=";")

In [202]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [203]:
path = f"{folder}/NetworkDesign_Nachfrage.csv"

In [204]:
nachfrage=pd.read_csv(path, sep=";")

In [205]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [206]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [207]:
I

['Charleston', 'Curitiba', 'Hamburg', 'Quanzhou', 'East London']

In [208]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [209]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika']

In [210]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [211]:
A

['klein', 'groß']

## Entscheidungsvariablen

In [212]:
infinity = solver.infinity()

In [213]:
# Definition der Entscheidungsvariablen
# x sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
x={}
for i in I:
  for j in J:
    x[i,j] = solver.IntVar(0, infinity, f"{i},{j}")

In [214]:
# Definition der Entscheidungsvariablen
# y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
y={}
for i in I:
  for a in A:
    y[i,a] = solver.BoolVar(f"{i},{a}")

In [215]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 35


## Parameter

In [216]:
# Herstell- und Transportkosten
produktionskosten.set_index(["Produktionsstandort","Markt"], inplace=True)

In [217]:
c = produktionskosten.to_dict("dict")["Produktionskosten"]

In [218]:
c

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

In [219]:
# Nachfrage aus den Märtken

In [220]:
nachfrage.set_index(["Markt"], inplace=True)

In [221]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [222]:
d

{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

In [223]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapazitaeten = kapa.set_index("Ausbaustufe")

In [224]:
cap = kapazitaeten.to_dict("dict")["Kapazitäten"]

In [225]:
cap

{'klein': 10, 'groß': 20}

In [226]:
f = kapa.set_index(["Produktionsstandort","Ausbaustufe"]).to_dict("dict")["Investitionen"]

In [227]:
f

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

## Zielfunktion

min $K=\sum_{i,j} c_{ij}* X_{ij} + \sum_{i,a} fc_{ia}* Y_{ia}$

In [228]:
# Maximierung der gesamten Deckungsbeiträge
solver.Minimize(sum(c[i,j]*x[i,j] for i in I for j in J) + sum(f[i,a]*y[i,a] for i in I for a in A))

## Nebenbedingungen

$\sum_i X_{ij} \ge d_j, ∀j$  

In [229]:
# Nachfrage in den Märkten erfüllen
for j in J:
  solver.Add(sum(x[i,j] for i in I)>=d[j])

$\sum_j X_{ij} \le \sum_a cap_{ia} * Y_{ia}, ∀i$

In [230]:
# Kapazitäten aufbauen und einhalten
for i in I:
  solver.Add(sum(x[i,j] for j in J)<= sum(cap[a]*y[i,a] for a in A))

## Berechnung Lösung

In [231]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
    print("\n")
    print("Produktions-Markt-Zuteilung")
    for j in J:
      print(f"Markt:{j}")
      for i in I:
        if x[i,j].solution_value()>0:
          print(f'{i,j} =', round(x[i,j].solution_value()))
    print("\n")
    print("Invest Standorte")
    for i in I:
      for a in A:
        if y[i,a].solution_value()>0:
          print(f'{i,a} =', round(y[i,a].solution_value()))
    print("\n")
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 23751.0


Produktions-Markt-Zuteilung
Markt:USA
('Curitiba', 'USA') = 12
Markt:Südamerika
('Curitiba', 'Südamerika') = 8
Markt:Europa
('Quanzhou', 'Europa') = 4
('East London', 'Europa') = 10
Markt:Asien
('Quanzhou', 'Asien') = 16
Markt:Afrika
('East London', 'Afrika') = 7


Invest Standorte
('Curitiba', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


